# Import libraries

In [2]:
pip install selenium pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#Import Packages
from selenium import webdriver
import time
import pandas as pd
import os

In [6]:
#Import Packages

from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

# Setup enviroment

In [7]:
url='https://www.linkedin.com/login'

In [15]:
#Load the web driver and get the url
# URL of the LinkedIn login page
url='https://www.linkedin.com/login'
# Path to ChromeDriver
chromedriver_path = r'C:\\Users\\My PC\\Downloads\\driver\\chromedriver-win64\\chromedriver.exe'

# Initialize Service and WebDriver
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

# Set implicit wait time and navigate to the URL
driver.implicitly_wait(10)
driver.get(url)

In [9]:
# Login credentials for LinkedIn
mail='daocuong4603@gmail.com'
password='Cuongcuong@03'

In [16]:
# Locate the email input field and enter the email and password
driver.find_element(By.XPATH,"//input[@type='text']").send_keys(mail)
driver.find_element(By.XPATH,"//input[@type='password']").send_keys(password)
# Locate and click the login button
driver.find_element(By.XPATH,"//button[@class='btn__primary--large from__button--floating']").click()


# Navigate to the LinkedIn job search page with specific keywords and location

In [17]:
driver.get('https://www.linkedin.com/jobs/search?keywords=Data&location=Vietnam&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0')

# Scarping linkedin job post

In [12]:
# Create empty DataFrame
df_jobs = pd.DataFrame()

# Create empty lists to add data
ljob_title = []
ljob_location = []
ljob_applicant= []
ljob_experience = []
ljob_description = []
ljob_skill = []
ljob_link = []


In [ ]:
# Iterate through the pages (1 to 20) of job listings because this time have 19 pages. So you need to check the end pages and replace your page number.
for j in range(1, 20):
    # Find and click on the page button
    page_num = driver.find_element(By.XPATH, f"//button[@aria-label='Page {j}']")
    page_num.click()
    time.sleep(3)  # Wait for the page to load

    # Retrieve all job listing elements on the current page
    job_block = driver.find_elements(By.XPATH, "//ul[@class='BqUOOnphjtSSwzavkqBDFBYMvNZaeSgndsgQ']/li")

    # Scroll each job element into view
    for job in job_block:
        driver.execute_script("arguments[0].scrollIntoView();", job)
        time.sleep(1)  # Pause to ensure content loads properly

    # Find all job links on the current page
    job_link = driver.find_elements(By.XPATH, "//ul[@class='BqUOOnphjtSSwzavkqBDFBYMvNZaeSgndsgQ']/li/div/div/div[1]/div[1]/div[2]/div[1]/a")

    # Access each job link to collect job details
    for link in job_link:
        link.click()
        time.sleep(3)  # Wait for the job details page to load

        # Extract job title
        try:
            job_title = driver.find_element(By.XPATH, "//h1[@class='t-24 t-bold inline']").text
        except:
            job_title = ""

        # Extract job location
        try:
            job_location = driver.find_element(By.XPATH, "//div[@class='job-details-jobs-unified-top-card__primary-description-container']/div/span").text
        except:
            job_location = ""

        # Extract job applicant
        try:
            job_applicant = driver.find_element(By.XPATH, "//div[@class='job-details-jobs-unified-top-card__primary-description-container']/div/span[5]").text
        except:
            job_applicant = ""
            
        # Extract job experience requirements
        try:
            job_experience = ', '.join(
                element.text.strip()
                for element in driver.find_elements(By.XPATH, "//div[@class='mt2 mb2']/ul/li[1]/span/span")
                if element.text.strip()
            )
        except:
            job_experience = ""

        # Extract required job skills
        try:
            job_skill = driver.find_element(By.XPATH, "//div[@class='display-flex flex-column overflow-hidden']/a").text
        except:
            job_skill = ""

        # Extract job description
        try:
            job_description = "\n".join(
                [job.text for job in driver.find_elements(By.XPATH, "//div[@class='mt4']/p/span")]
            )
        except:
            job_description = ""

        # Extract additional job links
        try:
            job_link = [link.get_attribute('href') for link in driver.find_elements(By.XPATH, "//div[@class='full-width artdeco-entity-lockup__title ember-view']/a")]
        except:
            job_link = []


        # Append extracted job details to respective lists
        ljob_title.append(job_title)
        ljob_location.append(job_location)
        ljob_applicant.append(job_applicant)
        ljob_experience.append(job_experience)
        ljob_skill.append(job_skill)
        ljob_description.append(job_description)
        ljob_link.append(job_link)


In [14]:
# Export to DataFrame
df_jobs = pd.DataFrame({
    "Job Title": ljob_title,
    "Location": ljob_location,
    "Applicant": ljob_applicant,
    "Experience": ljob_experience,
    "Description": ljob_description,
    "Skills": ljob_skill,
    "Link": ljob_link
    
})


df_jobs

,Job Title,Location,Applicant,Experience,Description,Skills,Link
0,Data Analysis,Ho Chi Minh City Metropolitan Area,Over 100 people clicked apply,,Responsibilities:\nAnalyze data and thoroughly...,,[https://www.linkedin.com/jobs/view/4099435056...
1,Data Engineer Intern,Hanoi Capital Region,11 applicants,,Job Title: Data Engineer Intern\nLocation: Han...,,[https://www.linkedin.com/jobs/view/4099435056...
2,Data Validation Specialist,"Ho Chi Minh City, Vietnam",59 applicants,,Meet our Client: Revolutionizing Insurance Dat...,,[https://www.linkedin.com/jobs/view/4099435056...
3,Data Science & AI Training & Internship,APJ,Over 100 applicants,,Refonte is a leading IT corporation globally r...,,[https://www.linkedin.com/jobs/view/4099435056...
4,Data Operations Specialist,"District 12, Ho Chi Minh City, Vietnam",31 applicants,,"About Concentrix\nConcentrix, a wholly owned s...",,[https://www.linkedin.com/jobs/view/4099435056...
...,...,...,...,...,...,...,...
124,Chuyên viên/Chuyên viên Cấp cao/Chuyên gia Phâ...,"Hanoi, Hanoi, Vietnam",1 person clicked apply,,\n\nNhiệm Vụ Chính\n Thực hiện triển khai mới ...,,[https://www.linkedin.com/jobs/view/4104042057...
125,Business analyst (exp Data),"Ho Chi Minh City, Vietnam",36 applicants,,Position: Business analyst\nLocation: District...,,[https://www.linkedin.com/jobs/view/4094830150...
126,Senior IT Business Analyst,Hanoi Capital Region,12 applicants,,I. Job Purpose\n- The job holder responsible f...,,[https://www.linkedin.com/jobs/view/4094830150...
127,Senior Business Data Analyst,Hanoi Capital Region,13 applicants,,1. Job Summary\n\nWe are looking for a Busines...,,[https://www.linkedin.com/jobs/view/4094830150...


In [118]:
df_jobs.to_csv('jobs.csv',encoding='utf-8-sig')

# Analysis

# Nháp

In [ ]:
# Tạo các danh sách trống để lưu thông tin công việc
ljob_titles = []
ljob_locations = []
ljob_experiences = []
ljob_descriptions = []

# Tìm tất cả các khối công việc
job_blocks = driver.find_elements(By.XPATH, "//ul[@class='BqUOOnphjtSSwzavkqBDFBYMvNZaeSgndsgQ']/li")

# Scroll từng khối công việc vào tầm nhìn để đảm bảo tải đủ nội dung
for job in job_blocks:
    driver.execute_script("arguments[0].scrollIntoView();", job)
    time.sleep(1)  # Đợi một chút để nội dung mới được load (nếu cần)

# Tìm tất cả các link công việc
job_links = driver.find_elements(By.XPATH, "//div[contains(@class, 'job-card-container')]//a[contains(@class, 'job-card-container__link')]")

# Lấy giá trị href của từng link
job_links = [link.get_attribute('href') for link in job_links]

# Duyệt qua từng link công việc
for link in job_links:
    # Truy cập vào từng link
    driver.get(link)
    time.sleep(3)  # Đợi trang công việc load xong

    try:
        # Lấy thông tin tiêu đề công việc
        job_title = driver.find_element(By.XPATH, "//h1[@class='t-24 t-bold inline']").text
    except Exception as e:
        job_title = "Không tìm thấy tiêu đề"
    
    try:
        # Lấy thông tin vị trí công việc
        job_location = driver.find_element(By.XPATH, "//span[@class='tvm__text tvm__text--low-emphasis']").text
    except Exception as e:
        job_location = "Không tìm thấy vị trí"

    try:
        # Lấy thông tin kinh nghiệm yêu cầu
        job_experience = driver.find_element(By.XPATH, "//li[@class='job-details-jobs-unified-top-card__job-insight job-details-jobs-unified-top-card__job-insight--highlight']//span").text
    except Exception as e:
        job_experience = "Không tìm thấy yêu cầu kinh nghiệm"

    try:
        # Lấy thông tin mô tả công việc
        job_description = driver.find_element(By.XPATH, "//div[@class='mt4']").text
    except Exception as e:
        job_description = "Không tìm thấy mô tả công việc"

    # Lưu thông tin vào các danh sách
    ljob_titles.append(job_title)
    ljob_locations.append(job_location)
    ljob_experiences.append(job_experience)
    ljob_descriptions.append(job_description)

# In ra kết quả thu thập được
print("Titles:", ljob_titles)
print("Locations:", ljob_locations)
print("Experiences:", ljob_experiences)
print("Descriptions:", ljob_descriptions)